

# Multi blade row meshing example
This basic example shows how to set up a multi blade row meshing instance and execute it in parallel.


## Perform required imports
Perform the required imports. It is assumed that the ``ansys-turbogrid-core``
package has been installed.



In [ ]:
import pathlib
import time

install_path = pathlib.PurePath(__file__).parent.parent.as_posix()

from ansys.turbogrid.core.multi_blade_row.multi_blade_row import MachineSizingStrategy
from ansys.turbogrid.core.multi_blade_row.multi_blade_row import multi_blade_row as MBR

all_face_areas = {}
all_element_counts = {}

## Create a multi_blade_row and initialize it
Use the Concepts NREC sample provided.



In [ ]:
start_time = time.time()
print(f"Start time: {time.asctime(time.localtime())}")
machine = MBR()
machine.init_from_ndf(
    f"{install_path}/tests/ndf/AxialFanMultiRow.ndf",
    # use_existing_tginit_cad=True, # Use this if you've already generated the CAD
)
# machine.plot_machine()
brs = machine.get_blade_row_names()
original_face_areas = machine.get_average_base_face_areas()
print(f"original_face_areas: {original_face_areas}")
machine.plot_machine()

## Set the sizing strategy for the machine
If this is omitted, the defaults for TurboGrid will be used.



In [ ]:
machine.set_machine_sizing_strategy(MachineSizingStrategy.MIN_FACE_AREA)

## Sweep a parameter
In this case, increase the global size factor, and measure the (base) layer face areas, and the element counts.



In [ ]:
for factor in [1, 1.5, 2]:
    all_face_areas[factor] = []
    all_element_counts[factor] = []
    machine.set_machine_size_factor(factor)
    all_face_areas[factor] = machine.get_average_base_face_areas()
    all_element_counts[factor] = machine.get_element_counts()

end_time = time.time()
print(f"End time: {time.asctime(time.localtime())}")
print(f"Duration: {(end_time-start_time)/60} minutes")
print(f"Average Base Face Sizes")
print(f"Blade Rows: {', '.join(brs)}")
for size_factor, face_areas in all_face_areas.items():
    print(
        f"""machine size_factor {size_factor}: {', '.join([f"{value:.6e}" for value in face_areas.values()])}  """
    )
print(f"Element Counts")
print(f"Blade Rows: {', '.join(brs)}")
for size_factor, element_counts in all_element_counts.items():
    print(
        f"""machine size_factor {size_factor}: {', '.join(str(value) for value in element_counts.values())}  """
    )